- LogisticRegression ROC-AUC SCORE:0.6514
- DecisionTreeClassifier ROC-AUC SCORE:  0.6788
- KNeighborsClassifier ROC-AUC SCORE:  0.5348
- VotingClassifier ROC-AUC SCORE:  0.7180
- RandomForestClassifier ROC-AUC SCORE:  0.8571
- AdaBoostClassifier ROC-AUC SCORE:  0.6767
- GradientBoostingClassifier ROC-AUC SCORE:  0.8622
- XGBoostClassifier ROC-AUC SCORE:  0.8487
- LightGBM ROC-AUC SCORE:  0.8558

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn = pd.read_csv('data/churn_ver02.csv')
target = churn.Exited
churn = churn.drop(columns='Exited')

In [3]:
#0.6788
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
dt = DecisionTreeClassifier()

DecisionTreeClassifier()

In [11]:
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}

In [20]:
grid_dt = GridSearchCV(dt, param_grid=param, 
                      scoring='roc_auc', cv=4,
                      n_jobs=-1, verbose=1)

In [21]:
grid_dt.fit(churn, target)

Fitting 4 folds for each of 98 candidates, totalling 392 fits


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 51)},
             scoring='roc_auc', verbose=1)

In [22]:
grid_dt.best_estimator_

DecisionTreeClassifier(max_depth=5)

In [23]:
grid_dt

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 51)},
             scoring='roc_auc', verbose=1)

- Final Ex

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(churn, target, test_size=0.3, stratify=target)

In [25]:
dt = DecisionTreeClassifier()
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}
grid_dt = GridSearchCV(dt, param_grid=param, 
                      scoring='roc_auc', cv=4,
                      n_jobs=-1, verbose=1)
grid_dt.fit(x_train, y_train)
print(grid_dt.best_estimator_)

Fitting 4 folds for each of 98 candidates, totalling 392 fits
DecisionTreeClassifier(criterion='entropy', max_depth=5)


In [29]:
#0.6788
pred = grid_dt.predict_proba(x_test)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.8398394441517621

In [32]:
from sklearn.model_selection import RandomizedSearchCV
dt = DecisionTreeClassifier()
param = {'criterion':['gini','entropy'],
         'max_depth':range(2,51)}
random_dt = RandomizedSearchCV(dt, param_distributions=param, scoring='roc_auc', n_iter=60, cv=4, n_jobs=-1, verbose=1)
random_dt.fit(x_train, y_train)
print(random_dt.best_estimator_)

Fitting 4 folds for each of 60 candidates, totalling 240 fits
DecisionTreeClassifier(max_depth=5)


In [33]:
#0.6788
pred = random_dt.predict_proba(x_test)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.8353555816039004